# Segmenting and Clustering Neighborhood in Toronto

### Part 1 Exploring Data and Cleaning

In [1]:
#importing pandas library
import pandas as pd

In [22]:
#using pandas read html to read the data from website
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

#assigning a variable to data points where Borough is 'Not assigned'
not_assigned = df[df['Borough']=='Not assigned'].index
not_assigned

#using the drop function to remove the data points where Borough is 'Not assigned' and reset index
df1 = df.drop(not_assigned).reset_index(drop=True)

df1

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [23]:
df1.shape

(103, 3)

 # 

### Part 2 Gecoder Segmenting

In [ ]:
!pip install geocoder

In [24]:
import geocoder
latitude=[]
longitude=[]
for code in df1['Postal Code']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    #print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        #print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])
    
df1['Latitude'] = latitude
df1['Longitude'] = longitude
    

In [25]:
df1

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.64869,-79.38544
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945


# 

### Part 3 Clustering Toronto Neighbourhood

In [9]:
pip install -U numpy #updating numpy
!conda install -c conda-forge folium=0.5.0 --yes #installing folium for map

     |████████████████████████████████| 15.3 MB 9.7 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibm-watson-machine-learning 1.0.45 requires pandas<=1.0.5, but you have pandas 1.2.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [26]:
import numpy as np

import folium # map rendering library

In [27]:
from geopy.geocoders import Nominatim 
address = 'Toronto, Ontario' # assign Toronto as address for map

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address) # geocode Toronto
latitude = location.latitude
longitude = location.longitude

# create map of Toronto using latitude and longitude values
map_tor = folium.Map(width=1000,height=500,location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor) 
    
map_tor